In [140]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import sklearn.linear_model
import string
#import pandas to load csv data 
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [141]:
data_directory = '/Users/fionalmeyda/Desktop/titanic/'
#read_csv allows you to control delimeters, column and row names
df_train = pd.read_csv(data_directory + 'train.csv')
df_test = pd.read_csv(data_directory + 'test.csv')

df_out =pd.read_csv(data_directory + 'gender_submission.csv')
labels_train = df_train['Survived']


In [142]:
#df_train.isna().sum()
#df_test.isna().sum()
#df_train
#df_test.columns

In [143]:
median_age = df_train['Age'].median()
#median age

median_fare =df_train['Fare'].median()
#median fare


In [144]:
title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']

In [145]:
import string
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if (str.find(big_string, substring) != -1):
            return substring
    print(big_string)
    return np.nan

In [146]:
def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

In [147]:
def preprocessing(df): 
    df = df[['Age','Pclass','Sex','SibSp','Parch','Fare','Name','Embarked']]
    
    #In this case, we impute missing data by replacing them with the median. The missing data for age and fare were replaced by its respectful median.
    df['Age'].fillna(median_age,inplace =True)
    df['Fare'].fillna(median_fare, inplace =True)
    
    #Replace name with proper titles
    df['Title']=df['Name'].map(lambda x: substrings_in_string(x, title_list))
    df['Title']=df.apply(replace_titles, axis=1)
    
    #New Column for Family Size
    df['Family_Size']=df['SibSp']+df['Parch']
    
    #combine age and class by multiplying them
    df['Age*Class']=df['Age']*df['Pclass']
    
    #fare per person
    df['Fare_Per_Person']=df['Fare']/(df['Family_Size']+1)
    
    #one hot encoding
    
    one_hot_title= pd.get_dummies(df['Title'])
    df =df.drop(['Title'],axis =1)
    df =df.join(one_hot_title)
    
    df = df.drop(['Name'], axis = 1)
    
    one_hot_embarked = pd.get_dummies(df['Embarked'])
    df =df.drop(['Embarked'],axis =1)
    df =df.join(one_hot_embarked)
    
    one_hot_gen = pd.get_dummies(df['Sex'])
    df =df.drop(['Sex'],axis =1)
    df =df.join(one_hot_gen)
    
    one_hot_class = pd.get_dummies(df['Pclass'])
    df =df.drop(['Pclass'],axis =1)
    df =df.join(one_hot_class)
    
    
    
    return df

In [148]:
df_train = preprocessing(df_train)
df_test = preprocessing(df_test)

/Users/fionalmeyda/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-147-5ea3d76fc6a7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Title']=df['Name'].map(lambda x: substrings_in_string(x, title_list))
<ipython-input-147-5ea3d76fc6a7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [149]:
df_test

,Age,SibSp,Parch,Fare,Family_Size,Age*Class,Fare_Per_Person,Master,Miss,Mr,Mrs,C,Q,S,female,male,1,2,3
0,34.5,0,0,7.8292,0,103.5,7.829200,0,0,1,0,0,1,0,0,1,0,0,1
1,47.0,1,0,7.0000,1,141.0,3.500000,0,0,0,1,0,0,1,1,0,0,0,1
2,62.0,0,0,9.6875,0,124.0,9.687500,0,0,1,0,0,1,0,0,1,0,1,0
3,27.0,0,0,8.6625,0,81.0,8.662500,0,0,1,0,0,0,1,0,1,0,0,1
4,22.0,1,1,12.2875,2,66.0,4.095833,0,0,0,1,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,28.0,0,0,8.0500,0,84.0,8.050000,0,0,1,0,0,0,1,0,1,0,0,1
414,39.0,0,0,108.9000,0,39.0,108.900000,0,0,1,0,1,0,0,1,0,1,0,0
415,38.5,0,0,7.2500,0,115.5,7.250000,0,0,1,0,0,0,1,0,1,0,0,1
416,28.0,0,0,8.0500,0,84.0,8.050000,0,0,1,0,0,0,1,0,1,0,0,1


In [150]:
#df_train.isna().sum()
#df_train.info()
df_test
#df_train['Age']
#df_test.head()

,Age,SibSp,Parch,Fare,Family_Size,Age*Class,Fare_Per_Person,Master,Miss,Mr,Mrs,C,Q,S,female,male,1,2,3
0,34.5,0,0,7.8292,0,103.5,7.829200,0,0,1,0,0,1,0,0,1,0,0,1
1,47.0,1,0,7.0000,1,141.0,3.500000,0,0,0,1,0,0,1,1,0,0,0,1
2,62.0,0,0,9.6875,0,124.0,9.687500,0,0,1,0,0,1,0,0,1,0,1,0
3,27.0,0,0,8.6625,0,81.0,8.662500,0,0,1,0,0,0,1,0,1,0,0,1
4,22.0,1,1,12.2875,2,66.0,4.095833,0,0,0,1,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,28.0,0,0,8.0500,0,84.0,8.050000,0,0,1,0,0,0,1,0,1,0,0,1
414,39.0,0,0,108.9000,0,39.0,108.900000,0,0,1,0,1,0,0,1,0,1,0,0
415,38.5,0,0,7.2500,0,115.5,7.250000,0,0,1,0,0,0,1,0,1,0,0,1
416,28.0,0,0,8.0500,0,84.0,8.050000,0,0,1,0,0,0,1,0,1,0,0,1


In [155]:
model1 = sk.linear_model.LogisticRegression()

In [156]:
model1.fit(df_train,labels_train)

/Users/fionalmeyda/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [158]:
y_pred =model1.predict(df_test)
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [159]:
df_out['Survived'] = y_pred
df_out.to_csv(data_directory + 'submit_kaggle.csv', index = False)